In [ ]:
!pip install datasets pandas transformers torch tables imbalanced-learn >/dev/null
!pip install --user simpletransformers >/dev/null

In [2]:
from datasets import load_dataset
silicone_merged = load_dataset("diwank/silicone-merged", "balanced")

train_df = silicone_merged["train"].to_pandas()
eval_df = silicone_merged["validation"].to_pandas()
test_df = silicone_merged["test"].to_pandas()

Reusing dataset silicone_merged_dataset (/home/jupyter/.cache/huggingface/datasets/diwank___silicone_merged_dataset/balanced/1.0.0/39a3b335ff82e7218cebbe6700a2bf12b187741b2768b59fc53a80ca49c89722)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import wandb
project = "da-silicone-combined"

# Turn off debug logs
import transformers
transformers.logging.set_verbosity_error()

# Turn off tokenizers parallelism
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# model configuration
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)

model_args = ClassificationArgs(
    n_gpu=2,
    num_train_epochs=4,
    learning_rate=3e-5,
    # train_batch_size=160,
    scheduler="polynomial_decay_schedule_with_warmup",
    polynomial_decay_schedule_power=2,
    evaluate_during_training=True,
    evaluate_during_training_verbose=True,
    evaluate_during_training_steps=2000,

    wandb_project=project,
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    overwrite_output_dir=True,
)

# Create a ClassificationModel
model = ClassificationModel("roberta", "distilroberta-base", args=model_args, num_labels=11)

# Train model
model.train_model(train_df, eval_df=eval_df)

In [ ]:
import pandas as pd

# Evaluate the model
combined = pd.concat([eval_df, test_df])
result, model_outputs, wrong_predictions = model.eval_model(combined)